In [1]:
!pip install --upgrade tensorflow-gpu
!pip install tensorflow-hub
!pip install seaborn
!pip install PyMuPdf

     |████████████████████████████████| 320.4MB 46kB/s 
     |████████████████████████████████| 6.3MB 5.9MB/s 


In [2]:
from os import listdir
from os.path import isfile, join
import sys, fitz

from google.colab import drive
drive.mount('/content/gdrive/')
import os

root_path = '/content/gdrive/My Drive/DS/search_doc/'
os.chdir(root_path)

Mounted at /content/gdrive/


In [3]:
import pandas as pd
import numpy as np
import os 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel

In [8]:
!mkdir use

In [10]:
os.chdir('use')

In [13]:
!curl -L -o 4.tar.gz "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed" 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  915M  100  915M    0     0  70.6M      0  0:00:12  0:00:12 --:--:-- 69.9M


In [14]:
!tar -xf 4.tar.gz

In [15]:
os.chdir('..')

In [21]:
model_path = 'use/'
model = hub.load(model_path)

def embed(input):
    return model(input)

In [22]:
from os import listdir
from os.path import isfile, join
import sys, fitz

files = [f for f in listdir(root_path) if isfile(join(root_path, f))]
documents = {}

for fname in files:
  doc = fitz.open(fname) 
  text = ""
  for page in doc:  
      text = page.getText().lower()  
  documents[fname] = text

In [35]:
df = pd.DataFrame({'name' : documents.keys() , 'content' : documents.values() })

In [36]:
ls =[]
chunksize =1000
le =len(df.content)
for i in range(0,le,chunksize):
    if(i+chunksize > le): 
        chunksize= le;
        ls.append(chunksize)
    else:
        a =i+chunksize
        ls.append(a)
ls
j=0
for i in ls:
    directory = "use/TrainModel/" + str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = "use/TrainModel/" + str(i)
    m=embed(df.content[j:i])
    exported_m = tf.train.Checkpoint(v=tf.Variable(m))
    exported_m.f = tf.function(
    lambda  x: exported_m.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
    tf.saved_model.save(exported_m,directory)
    j = i

INFO:tensorflow:Assets written to: use/TrainModel/16/assets


INFO:tensorflow:Assets written to: use/TrainModel/16/assets


In [37]:
df.drop('content', axis=1, inplace=True)
df.to_csv('use/index.csv',index=False)
ar =[]
for i in ls:
    directory = "use/TrainModel/" + str(i)
    if os.path.exists(directory):
        imported_m = tf.saved_model.load(directory)
        a= imported_m.v.numpy()
        ar.append(a)

cmodel = np.concatenate(ar)

In [38]:
def SearchDocument(query):
    q =[query]
    Q_Train =embed(q)
    df= pd.read_csv('use/index.csv')
    
    linear_similarities = linear_kernel(Q_Train, cmodel).flatten() 
    Top_index_doc = linear_similarities.argsort()[:-6:-1]
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'File_Name'] = df['name'][index] 
    for j,simScore in enumerate(linear_similarities[:-6:-1]):
        a.loc[j,'Score'] = simScore
    return a

In [39]:
SearchDocument('who  is good in analytics and studied in singapore')

,File_Name,Score
0,PWC_Olivia Peter_Regulatory Manager.pdf,0.332967
1,Xander_Rohit Khandelwal_VP.pdf,0.246875
2,PWC_Penny Lim_Risk AM.pdf,0.245234
3,Luminance Fund_Saleem Lalani_Research Associat...,0.183150
4,UOB AM_Felyna Lee_Product AVP.pdf,0.173067


In [41]:
SearchDocument('which item has acrylic acid in it?')

,File_Name,Score
0,SokalanCP5TechnicalInformation.pdf,0.170476
1,SokalanCP5FoodEU.pdf,0.160863
2,SokalanCP5ProductSpecification.pdf,0.157144
3,SokalanCP5REACHpolymers.pdf,0.152360
4,SokalanCP5NanoStatementFR.pdf,0.148385
